In [1]:
# necessary imports
from gensim.corpora import Dictionary
from gensim.models import LdaModel
from pprint import pprint
import pandas as pd
from nltk.corpus import stopwords
import re
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag, word_tokenize
import matplotlib.pyplot as plt
import nltk
from collections import defaultdict

In [2]:
#Functions used in LDA

def text_split_new(text) :

    #Initialize a set of frequent and not relevent words in english
    #stop_words = set(stopwords.words('english'))

    #Remove from text all punctuations
    text = re.sub(r'[^\w\s]', '', text)
    

    #Tokenise the text in words and convert in small letters
    words = word_tokenize(text)
    
    #Filter the words depending on their grammatical class
    
    text_splited = []

    for word in words:

        #Find the grammatical class (pos) of a word and tag it with it
        pos = pos_tag([word])[0][1]
        #Keep only the nouns (singular and plural)
        if pos in ['NN', 'NNS']: #and word not in stop_words:
            text_splited.append(word)
    
    return text_splited

def bag_of_words(text_splited) : 
    
    #Create a dictionary mapping each word to a unique id
    dictionary = Dictionary([text_splited])
    
    
    corpus = dictionary.doc2bow(text_splited)
    
    return corpus, dictionary

def lda(df_line):
    corpus, dictionary = df_line  # Unpacking the tuple directly
    lda_model = LdaModel([corpus], num_topics=1, id2word=dictionary)
    return lda_model

def get_topics(lda_model):
    
    return lda_model.show_topics(num_topics=1, num_words=50, formatted=True)

def extract(topics_tuple):
    topics = []
    pattern = re.compile(r'(\d+\.\d+)\*"(.*?)"')
    for topic_id, topics_str in topics_tuple:
        matches = pattern.findall(topics_str)
        for match in matches:
            freq = float(match[0])
            word = match[1]
            topics.append((word, freq))
    return topics

In [3]:
#First for loop to get the most used words for each decade

paths= ['data/wikipedia_timeline/1900s.txt', 'data/wikipedia_timeline/1910s.txt', 'data/wikipedia_timeline/1920s.txt', 'data/wikipedia_timeline/1930s.txt', 'data/wikipedia_timeline/1950s.txt', 'data/wikipedia_timeline/1960s.txt', 'data/wikipedia_timeline/1970s.txt', 'data/wikipedia_timeline/1980s.txt', 'data/wikipedia_timeline/1990s.txt', 'data/wikipedia_timeline/2000s.txt', 'data/wikipedia_timeline/2010s.txt']


for path in paths:
    decade_name = path.split('/')[-1].split('.')[0]
    decade=pd.read_csv(path, delimiter='\t', header=None)
    decade=decade.to_string(index=False, header=False)
    splitted_decade=text_split_new(decade)
    bag_decade=bag_of_words(splitted_decade)
    lda_decade=lda(bag_decade)
    print(f"Most used words for the decade {decade_name}: ",get_topics(lda_decade))
    print("the type of frequencies ", type(lda_decade.get_topics()))

Most used words for the decade 1900s:  [(0, '0.006*"World" + 0.005*"Machine" + 0.005*"Wayback" + 0.005*"Population" + 0.004*"History" + 0.003*"ISBN" + 0.003*"Data" + 0.003*"flight" + 0.003*"estimate" + 0.003*"Sheet" + 0.003*"machine" + 0.003*"Haub" + 0.002*"Empire" + 0.002*"States" + 0.002*"engine" + 0.002*"Fessenden" + 0.002*"m" + 0.002*"pp" + 0.002*"US" + 0.002*"Edison" + 0.002*"Company" + 0.002*"air" + 0.002*"de" + 0.002*"p" + 0.002*"decade" + 0.002*"Years" + 0.002*"Popular" + 0.002*"device" + 0.002*"html" + 0.002*"http" + 0.002*"Time" + 0.002*"Panama" + 0.001*"Great" + 0.001*"Worlds" + 0.001*"developed" + 0.001*"earthquake" + 0.001*"population" + 0.001*"car" + 0.001*"time" + 0.001*"production" + 0.001*"httpsAvebarchive" + 0.001*"paper" + 0.001*"seconds" + 0.001*"radio" + 0.001*"estimates" + 0.001*"Gabel" + 0.001*"use" + 0.001*"people" + 0.001*"flights" + 0.001*"built"')]
the type of frequencies  <class 'numpy.ndarray'>
Most used words for the decade 1910s:  [(0, '0.009*"World" + 0.

In [4]:
# Dictionary to store word frequencies for each decade
word_frequencies = defaultdict(lambda: defaultdict(int))

for path in paths:
    # Extract the decade from the file path
    decade_name = path.split('/')[-1].split('.')[0]
    
    # Read the file
    decade = pd.read_csv(path, delimiter='\t', header=None)
    decade = decade.to_string(index=False, header=False)
    
    # Process the text
    splitted_decade = text_split_new(decade)
    bag_decade = bag_of_words(splitted_decade)
    lda_decade = lda(bag_decade)
    
    # Get the topics and their frequencies
    topics_str = get_topics(lda_decade)
    # extract() function was written to extract the topics and their frequencies given their type
    topics = extract(topics_str)
    
    #print(f"Most used words for the decade {decade_name}: ", topics)
    # Update the word frequencies for each decade
    for word, freq in topics:
        word_frequencies[decade_name][word] = freq
        #print(word_frequencies)

# Convert the word frequencies dictionary to a DataFrame
df_word_frequencies = pd.DataFrame(word_frequencies).fillna(0)

# Transpose the DataFrame to have decades as rows and words as columns
df_word_frequencies = df_word_frequencies.T

# Display the DataFrame
print(df_word_frequencies.head())

#save in csv file
df_word_frequencies.to_csv("word_frequencies.csv")

       World  Machine  Wayback  Population  History   ISBN   Data  estimate  \
1900s  0.006    0.005    0.005       0.005    0.004  0.003  0.003     0.003   
1910s  0.009    0.000    0.000       0.000    0.000  0.002  0.000     0.000   
1920s  0.003    0.000    0.000       0.000    0.003  0.005  0.000     0.000   
1930s  0.005    0.000    0.000       0.000    0.002  0.004  0.000     0.000   
1950s  0.003    0.000    0.000       0.000    0.004  0.002  0.000     0.000   

       flight  Sheet  ...  CNN  damage  Event  Date  Country  toll   Al  \
1900s   0.003  0.003  ...  0.0     0.0    0.0   0.0      0.0   0.0  0.0   
1910s   0.000  0.000  ...  0.0     0.0    0.0   0.0      0.0   0.0  0.0   
1920s   0.000  0.000  ...  0.0     0.0    0.0   0.0      0.0   0.0  0.0   
1930s   0.000  0.000  ...  0.0     0.0    0.0   0.0      0.0   0.0  0.0   
1950s   0.000  0.000  ...  0.0     0.0    0.0   0.0      0.0   0.0  0.0   

       Typhoon  Description  Syria  
1900s      0.0          0.0    0.0  


In [5]:
# extract from the txt file the decades

summaries_per_decade = pd.read_csv('src/data/summaries_per_decade.txt', delimiter='\t', header=None)

summaries_1900s=summaries_per_decade[0][0]
summaries_1910s=summaries_per_decade[0][1]
summaries_1920s=summaries_per_decade[0][2]
summaries_1930s=summaries_per_decade[0][3]
summaries_1940s=summaries_per_decade[0][4]
summaries_1950s=summaries_per_decade[0][5]
summaries_1960s=summaries_per_decade[0][6]
summaries_1970s=summaries_per_decade[0][7]
summaries_1980s=summaries_per_decade[0][8]
summaries_1990s=summaries_per_decade[0][9]
summaries_2000s=summaries_per_decade[0][10]



In [ ]:
decades = [summaries_1900s,summaries_1910s,summaries_1920s,summaries_1930s,summaries_1940s,summaries_1950s,summaries_1960s,summaries_1970s,summaries_1980s,summaries_1990s,summaries_2000s]

decade_name = [1900,1910,1920,1930,1940,1950,1960,1970,1980,1990,2000]
i=0

for decade in decades:
    splitted_decade=text_split_new(decade)
    bag_decade=bag_of_words(splitted_decade)
    lda_decade=lda(bag_decade)
    print(f"Most used words for the decade {decade_name[i]}: ",get_topics(lda_decade))
    print("the type of frequencies ", type(lda_decade.get_topics()))
    i+=1

Most used words for the decade 1900:  [(0, '0.017*"film" + 0.008*"Pedro" + 0.007*"girl" + 0.006*"travelers" + 0.006*"astronomers" + 0.006*"Mr" + 0.005*"shot" + 0.005*"Jekyll" + 0.005*"train" + 0.005*"submarine" + 0.005*"Hyde" + 0.005*"man" + 0.005*"police" + 0.004*"Ned" + 0.004*"journey" + 0.004*"time" + 0.004*"Selenite" + 0.004*"front" + 0.004*"version" + 0.004*"scene" + 0.004*"woman" + 0.004*"space" + 0.004*"moves" + 0.004*"town" + 0.004*"help" + 0.004*"capsule" + 0.004*"sequence" + 0.004*"border" + 0.004*"ice" + 0.003*"gambler" + 0.003*"women" + 0.003*"cannon" + 0.003*"board" + 0.003*"customer" + 0.003*"suggest" + 0.003*"saloon" + 0.003*"mountains" + 0.003*"friends" + 0.003*"depicts" + 0.003*"shows" + 0.003*"sheriff" + 0.003*"vehicles" + 0.003*"falls" + 0.003*"revenge" + 0.003*"attacks" + 0.003*"proposes" + 0.003*"astronomer" + 0.003*"Selenites" + 0.003*"home" + 0.003*"rescuer"')]
the type of frequencies  <class 'numpy.ndarray'>
Most used words for the decade 1910:  [(0, '0.005*"man

In [10]:
# Dictionary to store word frequencies for each decade
word_frequencies_summaries = defaultdict(lambda: defaultdict(int))
i=0

for decade in decades:
    # Process the text
    splitted_decade = text_split_new(decade)
    bag_decade = bag_of_words(splitted_decade)
    lda_decade = lda(bag_decade)
    
    # Get the topics and their frequencies
    topics_str = get_topics(lda_decade)
    # extract() function was written to extract the topics and their frequencies given their type
    topics = extract(topics_str)
    
    #print(f"Most used words for the decade {decade_name}: ", topics)
    # Update the word frequencies for each decade
    for word, freq in topics:
        word_frequencies_summaries[decade_name[i]][word] = freq
    i+=1

# Convert the word frequencies dictionary to a DataFrame
df_word_frequencies = pd.DataFrame(word_frequencies_summaries).fillna(0)

# Transpose the DataFrame to have decades as rows and words as columns
df_word_frequencies = df_word_frequencies.T

# Display the DataFrame
print(df_word_frequencies.head())

#save in csv file
df_word_frequencies.to_csv("word_frequencies_summaries.csv")

       film  Pedro   girl  astronomers  travelers     Mr   shot    man  train  \
1900  0.017  0.008  0.007        0.006      0.006  0.006  0.005  0.005  0.005   
1910  0.004  0.000  0.004        0.000      0.000  0.000  0.000  0.005  0.000   
1920  0.004  0.000  0.002        0.000      0.000  0.000  0.000  0.005  0.000   
1930  0.002  0.000  0.000        0.000      0.000  0.002  0.000  0.003  0.000   
1940  0.002  0.000  0.000        0.000      0.000  0.000  0.000  0.003  0.001   

      Jekyll  ...  discovers  friends  meet  kills  school  body  attempts  \
1900   0.005  ...        0.0      0.0   0.0    0.0     0.0   0.0       0.0   
1910   0.000  ...        0.0      0.0   0.0    0.0     0.0   0.0       0.0   
1920   0.000  ...        0.0      0.0   0.0    0.0     0.0   0.0       0.0   
1930   0.000  ...        0.0      0.0   0.0    0.0     0.0   0.0       0.0   
1940   0.000  ...        0.0      0.0   0.0    0.0     0.0   0.0       0.0   

      reveals  team  people  
1900      0.0 